In [1]:
import keras
import keras.backend as K
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.callbacks import CSVLogger
import os
import keras.activations
import time

C:\Users\MaksutovRN\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

data_mean = round(x_train.mean(), 2)
print(data_mean)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples
0.13


In [7]:
batch_size = 64
epochs = 100
units = 64
experiments = 5
start = 0
# activations = ['sigmoid', 'tanh', 'relu', 'linear', 'elu', 'softplus', 'softsign', 'hard_sigmoid', 'LeakyReLU', 'ThresholdedReLU']
activations = ['ThresholdedReLU']
# activations = ['selu']

In [8]:
start_time = time.time()
counter = 0
total_items = len(activations) * experiments

for act in activations:
    print("Training for activation " + act + " with RMSProp optimizer")

    for i in range(experiments):

        K.clear_session()
        K.reset_uids()

        act_dict = {
          'sigmoid': Activation(keras.activations.sigmoid),
          'tanh': Activation(keras.activations.tanh),
          'relu': Activation(keras.activations.relu),
          'linear': Activation(keras.activations.linear),
          'elu': Activation(keras.activations.elu),
          'softplus': Activation(keras.activations.softplus),
          'softsign': Activation(keras.activations.softsign),
          'hard_sigmoid': Activation(keras.activations.hard_sigmoid),
          'LeakyReLU': keras.layers.advanced_activations.LeakyReLU(alpha=0.01),
          'selu': Activation(keras.activations.selu),
          'ThresholdedReLU': keras.layers.advanced_activations.ThresholdedReLU(theta=data_mean) 
        }

        model_name = 'normd_' + act + '_' + str(data_mean) + '_rmsp_' + str(i + start) + '_' + str(units)
        inputs = Input(shape=(784,))
        x = Dense(units)(inputs)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        x = Dense(units)(x)
        x = act_dict[act](x)
        x = Dropout(0.2)(x)
        predictions = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

        model.compile(loss='categorical_crossentropy',
                          optimizer=RMSprop(lr=0.001),
                          metrics=['accuracy'])

        csv_logger = CSVLogger('./training_logs/%s.csv' % (model_name), append=False)
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test, y_test), callbacks=[csv_logger])

        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        t = time.time()
        time_diff = t - start_time
        counter +=1
        rem_items = total_items - counter
        total_time = round((total_items / counter) * time_diff)
        rem_time = round(total_time - time_diff)
        m, s = divmod(rem_time, 60)
        h, m = divmod(m, 60)
        d, h = divmod(h, 24)
        print('Remaining time: %d days %d hours %02d minutes %02d seconds' % (d, h, m, s))

Training for activation ThresholdedReLU with RMSProp optimizer
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.4747 - acc: 0.8564 - val_loss: 0.1769 - val_acc: 0.9467
Epoch 2/100
60000/60000 [==============================] - 3s 43us/step - loss: 0.2344 - acc: 0.9295 - val_loss: 0.1379 - val_acc: 0.9561
Epoch 3/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.1943 - acc: 0.9437 - val_loss: 0.1341 - val_acc: 0.9591
Epoch 4/100
60000/60000 [==============================] - 3s 42us/step - loss: 0.1743 - acc: 0.9493 - val_loss: 0.1144 - val_acc: 0.9670
Epoch 5/100
60000/60000 [==============================] - 2s 41us/step - loss: 0.1603 - acc: 0.9539 - val_loss: 0.1224 - val_acc: 0.9658
Epoch 6/100
60000/60000 [==============================] - 3s 44us/step - loss: 0.1517 - acc: 0.9563 - val_loss: 0.1083 - val_acc: 0.9683
Epoch 7/100
60000/60000 [==============================] - 

60000/60000 [==============================] - 2s 39us/step - loss: 0.1352 - acc: 0.9700 - val_loss: 0.1720 - val_acc: 0.9704
Epoch 60/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.1342 - acc: 0.9702 - val_loss: 0.1775 - val_acc: 0.9707
Epoch 61/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.1360 - acc: 0.9713 - val_loss: 0.1747 - val_acc: 0.9721
Epoch 62/100
60000/60000 [==============================] - 2s 39us/step - loss: 0.1384 - acc: 0.9704 - val_loss: 0.1678 - val_acc: 0.9715
Epoch 63/100
60000/60000 [==============================] - 2s 36us/step - loss: 0.1361 - acc: 0.9704 - val_loss: 0.1692 - val_acc: 0.9712
Epoch 64/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.1346 - acc: 0.9709 - val_loss: 0.1764 - val_acc: 0.9723
Epoch 65/100
60000/60000 [==============================] - 2s 37us/step - loss: 0.1374 - acc: 0.9720 - val_loss: 0.1845 - val_acc: 0.9709
Epoch 66/100
60000/60000 [==============

Epoch 17/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1254 - acc: 0.9671 - val_loss: 0.1284 - val_acc: 0.9744
Epoch 18/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1204 - acc: 0.9690 - val_loss: 0.1350 - val_acc: 0.9739
Epoch 19/100
60000/60000 [==============================] - 4s 67us/step - loss: 0.1217 - acc: 0.9686 - val_loss: 0.1268 - val_acc: 0.9727
Epoch 20/100
60000/60000 [==============================] - 4s 62us/step - loss: 0.1232 - acc: 0.9683 - val_loss: 0.1376 - val_acc: 0.9745
Epoch 21/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1265 - acc: 0.9677 - val_loss: 0.1322 - val_acc: 0.9754
Epoch 22/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1248 - acc: 0.9688 - val_loss: 0.1301 - val_acc: 0.9737
Epoch 23/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1238 - acc: 0.9701 - val_loss: 0.1271 - val_acc: 0.9751
Epoch 24/100
60000/60000 [=

60000/60000 [==============================] - 4s 65us/step - loss: 0.1398 - acc: 0.9714 - val_loss: 0.1979 - val_acc: 0.9711
Epoch 76/100
60000/60000 [==============================] - 4s 68us/step - loss: 0.1372 - acc: 0.9722 - val_loss: 0.1866 - val_acc: 0.9712
Epoch 77/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1430 - acc: 0.9725 - val_loss: 0.1915 - val_acc: 0.9703
Epoch 78/100
60000/60000 [==============================] - 3s 53us/step - loss: 0.1387 - acc: 0.9725 - val_loss: 0.1937 - val_acc: 0.9708
Epoch 79/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1451 - acc: 0.9712 - val_loss: 0.1918 - val_acc: 0.9725
Epoch 80/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1389 - acc: 0.9726 - val_loss: 0.1958 - val_acc: 0.9720
Epoch 81/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1423 - acc: 0.9730 - val_loss: 0.1871 - val_acc: 0.9727
Epoch 82/100
60000/60000 [==============

Epoch 33/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1292 - acc: 0.9700 - val_loss: 0.1445 - val_acc: 0.9713
Epoch 34/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1262 - acc: 0.9696 - val_loss: 0.1450 - val_acc: 0.9709
Epoch 35/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.1211 - acc: 0.9710 - val_loss: 0.1562 - val_acc: 0.9736
Epoch 36/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1277 - acc: 0.9691 - val_loss: 0.1444 - val_acc: 0.9729
Epoch 37/100
60000/60000 [==============================] - 3s 51us/step - loss: 0.1271 - acc: 0.9712 - val_loss: 0.1579 - val_acc: 0.9733
Epoch 38/100
60000/60000 [==============================] - 3s 52us/step - loss: 0.1266 - acc: 0.9705 - val_loss: 0.1531 - val_acc: 0.9739
Epoch 39/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1262 - acc: 0.9713 - val_loss: 0.1456 - val_acc: 0.9738
Epoch 40/100
60000/60000 [=

Epoch 92/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1400 - acc: 0.9733 - val_loss: 0.1950 - val_acc: 0.9703
Epoch 93/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1498 - acc: 0.9727 - val_loss: 0.2037 - val_acc: 0.9717
Epoch 94/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1486 - acc: 0.9721 - val_loss: 0.1944 - val_acc: 0.9702
Epoch 95/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1503 - acc: 0.9721 - val_loss: 0.1874 - val_acc: 0.9705
Epoch 96/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1446 - acc: 0.9720 - val_loss: 0.1911 - val_acc: 0.9725
Epoch 97/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1499 - acc: 0.9726 - val_loss: 0.1950 - val_acc: 0.9705
Epoch 98/100
60000/60000 [==============================] - 3s 49us/step - loss: 0.1504 - acc: 0.9717 - val_loss: 0.1956 - val_acc: 0.9717
Epoch 99/100
60000/60000 [=

Epoch 50/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1311 - acc: 0.9718 - val_loss: 0.1741 - val_acc: 0.9699
Epoch 51/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1308 - acc: 0.9713 - val_loss: 0.1895 - val_acc: 0.9722
Epoch 52/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1317 - acc: 0.9710 - val_loss: 0.1663 - val_acc: 0.9729
Epoch 53/100
60000/60000 [==============================] - 3s 47us/step - loss: 0.1297 - acc: 0.9718 - val_loss: 0.1705 - val_acc: 0.9725
Epoch 54/100
60000/60000 [==============================] - 3s 46us/step - loss: 0.1377 - acc: 0.9710 - val_loss: 0.1885 - val_acc: 0.9712
Epoch 55/100
60000/60000 [==============================] - 3s 48us/step - loss: 0.1325 - acc: 0.9719 - val_loss: 0.1802 - val_acc: 0.9731
Epoch 56/100
60000/60000 [==============================] - 3s 50us/step - loss: 0.1350 - acc: 0.9716 - val_loss: 0.1661 - val_acc: 0.9717
Epoch 57/100
60000/60000 [=

Epoch 8/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1397 - acc: 0.9605 - val_loss: 0.1124 - val_acc: 0.9703
Epoch 9/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1331 - acc: 0.9622 - val_loss: 0.1145 - val_acc: 0.9712
Epoch 10/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1352 - acc: 0.9631 - val_loss: 0.1146 - val_acc: 0.9725
Epoch 11/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1289 - acc: 0.9639 - val_loss: 0.1231 - val_acc: 0.9706
Epoch 12/100
60000/60000 [==============================] - 4s 61us/step - loss: 0.1269 - acc: 0.9649 - val_loss: 0.1249 - val_acc: 0.9718
Epoch 13/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1252 - acc: 0.9660 - val_loss: 0.1169 - val_acc: 0.9713
Epoch 14/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1272 - acc: 0.9656 - val_loss: 0.1190 - val_acc: 0.9726
Epoch 15/100
60000/60000 [===

Epoch 67/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1373 - acc: 0.9720 - val_loss: 0.1862 - val_acc: 0.9721
Epoch 68/100
60000/60000 [==============================] - 3s 54us/step - loss: 0.1328 - acc: 0.9720 - val_loss: 0.1854 - val_acc: 0.9730
Epoch 69/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1398 - acc: 0.9709 - val_loss: 0.1829 - val_acc: 0.9718
Epoch 70/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1369 - acc: 0.9730 - val_loss: 0.1938 - val_acc: 0.9703
Epoch 71/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.1426 - acc: 0.9725 - val_loss: 0.1881 - val_acc: 0.9717
Epoch 72/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.1352 - acc: 0.9726 - val_loss: 0.1892 - val_acc: 0.9708
Epoch 73/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1407 - acc: 0.9718 - val_loss: 0.1848 - val_acc: 0.9725
Epoch 74/100
60000/60000 [=